## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
model_Xception=keras.applications.Xception(include_top=False, weights='imagenet',
                                           input_tensor=input_tensor,
                                           pooling=None, classes=10)

'''Resnet 50 架構'''
model_Resnet50=keras.applications.ResNet50(include_top=False, weights='imagenet',
                                           input_tensor=input_tensor,
                                           pooling=None, classes=10)



Using TensorFlow backend.


C:\Users\wjr63\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [2]:
model_Xception.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [3]:
model_Resnet50.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

## 添加層數

In [4]:
x = model_Xception.output

'''可以參考Cifar10實作章節'''
predictions = Flatten()(x)
predictions = Dense(output_dim=10,activation='softmax')(predictions)
model_Xception = Model(inputs=model_Xception.input, outputs=predictions)
print('Model_Xception深度：', len(model_Xception.layers))


Model_Xception深度： 134


C:\Users\wjr63\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  """


In [5]:
x = model_Resnet50.output

'''可以參考Cifar10實作章節'''
predictions = Flatten()(x)
predictions = Dense(output_dim=10,activation='softmax')(predictions)
model_Resnet50 = Model(inputs=model_Resnet50.input, outputs=predictions)
print('Model_Resnet50深度：', len(model_Resnet50.layers))

Model_Resnet50深度： 177


C:\Users\wjr63\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  """


## 鎖定特定幾層不要更新權重

In [6]:
for layer in model_Xception.layers[:100]:
    layer.trainable = False
for layer in model_Xception.layers[100:]:
    layer.trainable = True

In [7]:
for layer in model_Resnet50.layers[:100]:
    layer.trainable = False
for layer in model_Resnet50.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [8]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


C:\Users\wjr63\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Training

In [9]:
# compile the model (should be done *after* setting layers to non-trainable)

model_Xception.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model_Xception.fit(x_train,y_train,batch_size=32,epochs=100)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
50000/50000 [==============================] - 68s 1ms/step - loss: 1.4624 - acc: 0.4755
Epoch 2/100
50000/50000 [==============================] - 65s 1ms/step - loss: 0.9823 - acc: 0.6607
Epoch 3/100
50000/50000 [==============================] - 65s 1ms/step - loss: 0.8692 - acc: 0.6972
Epoch 4/100
50000/50000 [==============================] - 65s 1ms/step - loss: 0.7774 - acc: 0.7287
Epoch 5/100
50000/50000 [==============================] - 64s 1ms/step - loss: 0.7036 - acc: 0.7563
Epoch 6/100
50000/50000 [==============================] - 64s 1ms/step - loss: 0.6291 - acc: 0.7790
Epoch 7/100
50000/50000 [==============================] - 65s 1ms/step - loss: 0.5525 - acc: 0.8062
Epoch 8/100
50000/50000 [==============================] - 66s 1ms/step - loss: 0.4931 - acc: 0.8274
Epoch 9/100
50000/50000 [==============================] - 66s 1ms/step - loss: 0.4452 - acc: 0.8

50000/50000 [==============================] - 62s 1ms/step - loss: 0.0629 - acc: 0.9792
Epoch 77/100
50000/50000 [==============================] - 62s 1ms/step - loss: 0.0655 - acc: 0.9774
Epoch 78/100
50000/50000 [==============================] - 62s 1ms/step - loss: 0.0648 - acc: 0.9780
Epoch 79/100
50000/50000 [==============================] - 62s 1ms/step - loss: 0.0575 - acc: 0.9808
Epoch 80/100
50000/50000 [==============================] - 62s 1ms/step - loss: 0.0597 - acc: 0.9801
Epoch 81/100
50000/50000 [==============================] - 62s 1ms/step - loss: 0.0599 - acc: 0.9801
Epoch 82/100
50000/50000 [==============================] - 62s 1ms/step - loss: 0.0643 - acc: 0.9777
Epoch 83/100
50000/50000 [==============================] - 62s 1ms/step - loss: 0.0608 - acc: 0.9792
Epoch 84/100
50000/50000 [==============================] - 62s 1ms/step - loss: 0.0584 - acc: 0.9803
Epoch 85/100
50000/50000 [==============================] - 62s 1ms/step - loss: 0.0571 - acc: 

In [10]:
# compile the model (should be done *after* setting layers to non-trainable)

model_Resnet50.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model_Resnet50.fit(x_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
50000/50000 [==============================] - 123s 2ms/step - loss: 1.6711 - acc: 0.5487
Epoch 2/100
50000/50000 [==============================] - 118s 2ms/step - loss: 1.0509 - acc: 0.6630
Epoch 3/100
50000/50000 [==============================] - 118s 2ms/step - loss: 1.0509 - acc: 0.6895
Epoch 4/100
50000/50000 [==============================] - 118s 2ms/step - loss: 0.8546 - acc: 0.7365
Epoch 5/100
50000/50000 [==============================] - 118s 2ms/step - loss: 0.8836 - acc: 0.7541
Epoch 6/100
50000/50000 [==============================] - 118s 2ms/step - loss: 0.8242 - acc: 0.7647
Epoch 7/100
50000/50000 [==============================] - 118s 2ms/step - loss: 0.6254 - acc: 0.8063
Epoch 8/100
50000/50000 [==============================] - 118s 2ms/step - loss: 0.5231 - acc: 0.8417
Epoch 9/100
50000/50000 [==============================] - 118s 2ms/step - loss: 0.6547 - acc: 0.8500
Epoch 10/100
50000/50000 [==============================] - 118s 2ms/step - loss: 

50000/50000 [==============================] - 118s 2ms/step - loss: 0.1267 - acc: 0.9754
Epoch 81/100
50000/50000 [==============================] - 118s 2ms/step - loss: 0.1094 - acc: 0.9770
Epoch 82/100
50000/50000 [==============================] - 118s 2ms/step - loss: 0.1075 - acc: 0.9783
Epoch 83/100
50000/50000 [==============================] - 118s 2ms/step - loss: 0.1179 - acc: 0.9767
Epoch 84/100
50000/50000 [==============================] - 118s 2ms/step - loss: 0.1103 - acc: 0.9774
Epoch 85/100
50000/50000 [==============================] - 118s 2ms/step - loss: 0.1145 - acc: 0.9769
Epoch 86/100
50000/50000 [==============================] - 118s 2ms/step - loss: 0.1023 - acc: 0.9772
Epoch 87/100
50000/50000 [==============================] - 118s 2ms/step - loss: 0.1017 - acc: 0.9781
Epoch 88/100
50000/50000 [==============================] - 118s 2ms/step - loss: 0.0982 - acc: 0.9786
Epoch 89/100
50000/50000 [==============================] - 118s 2ms/step - loss: 0.09